In [10]:
from bs4 import BeautifulSoup as bs
#from sklearn.cluster import KMeans
import requests


In [11]:
#price, locali, superficie, bagni, piano
#https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1
content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=1")
'''
EXAMPLE to use while scraping until we have the desired number of entries
while df.cols < 10000:
    i +=1
    content = requests.get("https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i))
    #do extraction of needed details here
    #add details to df
    '''


In [19]:
soup = bs(content.text, 'html.parser')


In [77]:
first_app = soup.find_all(class_="listing-features list-piped")[0]
print(first_app.prettify())
#getting the stuff of the first apartement
#price is contained in the class "lif__item lif__pricing"
'''
every other detail is contained after this:  
    <li class="lif__item">
      <div class="lif__data">
'''

<ul class="listing-features list-piped">
 <li class="lif__item lif__pricing">
  <span style="font-size: 16px;">
   da
  </span>
  € 192.000
 </li>
 <li class="lif__item">
  <div class="lif__data">
   <span class="text-bold">
    1-5
   </span>
   <svg class="icon-svg" height="16" width="16">
    <use xlink:href="#rooms">
    </use>
   </svg>
  </div>
  <div class="lif__text lif--muted">
   locali
  </div>
 </li>
 <li class="lif__item">
  <div class="lif__data">
   da
   <span class="text-bold">
    46
   </span>
   m
   <sup>
    2
   </sup>
  </div>
  <div class="lif__text lif--muted">
   superficie
  </div>
 </li>
 <li class="lif__item lif__guaranteed">
  <div class="lif__text lif--muted">
   immobile
  </div>
  <div class="lif__text text-bold text-danger">
   garantito
  </div>
 </li>
</ul>



'\nevery other detail is contained after this:  \n    <li class="lif__item">\n      <div class="lif__data">\n'

In [72]:
i = 0
for details in soup.find_all(class_="listing-features list-piped"):
    price = details.contents[1]
    i+=1 
    print("\n \n _____ Apartement "+str(i)+" _____")
    print("\n" + str(price))
    
print(i)
   
'''
I counted the number of listings per page. Its 25. So it seems this is somehow working :)
'''            
#<ul class="listing-features list-piped"> contains the desired information


 
 _____ Apartement 1 _____

<li class="lif__item lif__pricing"><span style="font-size: 16px;">da </span>€ 192.000
                </li>

 
 _____ Apartement 2 _____

<li class="lif__item lif__pricing">
                    € 225.000
                </li>

 
 _____ Apartement 3 _____

<li class="lif__item lif__pricing">
                    € 520.000
                </li>

 
 _____ Apartement 4 _____

<li class="lif__item lif__pricing">
                    € 579.000
                </li>

 
 _____ Apartement 5 _____

<li class="lif__item lif__pricing">
                    € 1.380.000
                </li>

 
 _____ Apartement 6 _____

<li class="lif__item lif__pricing">
                    € 450.000
                </li>

 
 _____ Apartement 7 _____

<li class="lif__item lif__pricing"><div>€ 159.000</div><div class="features__price-old"><span class="features__price-old--price">€ 169.000</span><span class="features__price-old--discount">(-5.9%)</span></div></li>

 
 _____ Apartement 8 __

'\nI counted the number of listings per page. Its 25. So it seems this is somehow working :)\n'

In [80]:
detl ={}
i = 1
for details in soup.find_all(class_="lif__data"):
    det = details.contents[0]
    i +=1 
    detl[i] = det
print(i)
#<ul class="listing-features list-piped"> contains the desired information
'''
This doesn't work really good so far, but its getting the details. Unfortunately a lot of shit as well. i is 186 and should be 25
'''

IndentationError: unexpected indent (<ipython-input-80-b5aceb3ca62b>, line 9)